In [12]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [13]:
"""
    双低筛选策略，参考：
    https://xueqiu.com/1314783718/166213808
    https://www.jisilu.cn/question/408977
    https://www.jisilu.cn/question/273614
"""

import bisect
import copy
import logging
import math
from statistics import mean
import pandas as pd
from jqdata import get_all_trade_days
from jqdata import bond
from datetime import datetime, timedelta

logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s', level=logging.DEBUG)

# 低于110元的转债我们看作低估
UNDERRATE_PRICE = 110

# 聚宽的可转债数据从2018-09-13开始记录
JQDATA_BEGIN_DATE = '2018-09-13'

        
class ConvertBondBeta(object):
    """得到指定时间的可转债基本信息，包括:
        市场总量
        低估转债(<110元)市场总量
        全部转债价格算术平均值
        全部转债溢价率算术平均值
        
        指定一个时间段，算出当前转债市场指标的历史百分位
    """
    
    def __init__(self, base_date=None, history_days=365*3):
        """
        input:
            base_date: 查询时间，格式为'yyyy-MM-dd'，默认为当天
            history_days: 默认历史区间位前3年
        """
        if not base_date:
            self._base_date = datetime.now().date()
        else:
            self._base_date = datetime.strptime(base_date, '%Y-%m-%d')
            
        self._begin_date = self._base_date - timedelta(history_days)
        self._end_date = self._base_date
        
        if self._begin_date < datetime.strptime(JQDATA_BEGIN_DATE, '%Y-%m-%d'):
            self._begin_date = datetime.strptime(JQDATA_BEGIN_DATE, '%Y-%m-%d')
        
        self._begin_date = self._begin_date.strftime('%Y-%m-%d')
        self._end_date = self._end_date.strftime('%Y-%m-%d')
        self._base_date = self._base_date.strftime('%Y-%m-%d')
        
    def get_bonds(self, date=None):
        """
        获取指定日期的可转债市场正常存续的转债的基本信息
        
            code: 可转债代码
            short_name: 可转债名称
            raise_fund_volume: 发行总量(元)
            current_fund_volume: 当前存量总量(元)
            price: 收盘价格
            day_market_volume: 当日市场成交总价
            convert_premium_ratio: 转股溢价率
            convert_price: 转股价格 (如果没有下修的话就是约定转股价)
            stock_price: 正股价格
            last_cash_date: 最终兑付日(到期时间)
            double_low: 转债价格+溢价率X100
            ytm: 计算方式比较复杂，暂缺失
        """
        
        if date is None:
            date = datetime.strptime(self._base_date, '%Y-%m-%d').date()
        else:
            date = datetime.strptime(date, '%Y-%m-%d').date()
        
        df_bonds = bond.run_query(
                query(bond.CONBOND_BASIC_INFO).filter(
                                    bond.CONBOND_BASIC_INFO.bond_type_id == 703013,
                                    bond.CONBOND_BASIC_INFO.list_status_id.in_(['301001', '301099']),
                                    bond.CONBOND_BASIC_INFO.interest_begin_date < date,
                                    bond.CONBOND_BASIC_INFO.last_cash_date >= date
                                ).order_by('code').limit(10000)
            )        
    
        bond_list = []
        
        total_market = 0.0
        for index, row in df_bonds.iterrows():            
            bond_info = {
                        'code': row['code'], 
                        'short_name': row['short_name'], 
                        'stock_code': row['company_code'],
                        'convert_price': row['convert_price'],
                        'last_cash_date': row['last_cash_date']
            }
            
            if str(row['list_status_id']) == '301099':
                # issue-2: CONBOND_BASIC_INO表中数据更新不及时，需要去BOND_BASIC_INFO中确认一下
                bond_basic_info = bond.run_query(
                                query(bond.BOND_BASIC_INFO).filter(
                                    bond.BOND_BASIC_INFO.code == row['code']
                                )
                              )            
                if str(bond_basic_info['list_status_id'][0]) == '301099':
                    # 未上市
                    continue            
            
            if row['list_date'] and row['list_date'] > date:
                # 只发布了信息，还没有正式上市，暂不记入
                continue
            
            
            # 发行总量(万元)
            if not math.isnan(row['actual_raise_fund']):
                bond_info['raise_fund_volume'] = float(row['actual_raise_fund']) * 10000
            else:
                bond_info['raise_fund_volume'] = float(row['plan_raise_fund']) * 10000
            bond_info['current_fund_volume'] = bond_info['raise_fund_volume']
            
            # 转股信息
            bond_stock = bond.run_query(
                query(bond.CONBOND_DAILY_CONVERT).filter(
                                    bond.CONBOND_DAILY_CONVERT.code == bond_info['code'], 
                                    bond.CONBOND_DAILY_CONVERT.date <= date,
                                )
            )   
            
            # 统计转股信息，如果99%转股，就代表强赎退市，暂不记入，另外要修正存量债券数目
            if not bond_stock['acc_convert_ratio'].empty:
                if bond_stock['acc_convert_ratio'].iloc[-1] >= 99.5:
                    continue
                else:
                    bond_info['current_fund_volume'] = bond_info['raise_fund_volume'] * \
                            (100.0 - bond_stock['acc_convert_ratio'].iloc[-1]) / 100.0
            
            # 先取得转股价，然后取得正股收盘价，然后计算溢价率：转股溢价=（100/转股价格）*正股收盘价-可转债收盘价）
            # 转股价如果有下修，先取得下修转股价
            if not bond_stock['convert_price'].empty:
                bond_info['convert_price'] =  float(bond_stock['convert_price'].iloc[-1])                
            
            # 当前市场收盘价格
            bond_market = bond.run_query(
                query(bond.CONBOND_DAILY_PRICE).filter(
                                    bond.CONBOND_DAILY_PRICE.code == bond_info['code'], 
                                    bond.CONBOND_DAILY_PRICE.date <= date,
                                )
            )
            try:
                bond_info['price'] =  float(bond_market['close'].iloc[-1])
                bond_info['day_market_volume'] = float(bond_market['money'].iloc[-1])
            except Exception:
                # 有部分还没有公布信息的先跳过
                bond_info['price'] =  float(row['par'])
                continue
            
            if bond_info['price'] < 1:
                # 停牌
                continue            

                
            # 获取正股价格
            df_stock_price = get_price(bond_info['stock_code'],
                                       count = 7,
                                       end_date= date,
                                       frequency='daily', 
                                       fields=['close'])
            bond_info['stock_price'] = df_stock_price['close'][-1]
            bond_info['convert_premium_ratio'] = (bond_info['price'] - 100/bond_info['convert_price']*bond_info['stock_price']) / \
                                                 (100/bond_info['convert_price']*bond_info['stock_price'])
            bond_info['double_low'] = bond_info['price'] + bond_info['convert_premium_ratio'] * 100
                      
            bond_list.append(bond_info)
            
        bond_list = sorted(bond_list, key=lambda x: x['double_low'])
        return bond_list
        
            
    def get_bonds_factors(self, bond_list=None):
        """
        获取当前时间的市场总量, 低估转债市场总量，指数平均价格，指数平均溢价率

        output:
             (total_market(元), underrate_market, avg_price, avg_premium_ratio)
        """
        
        if bond_list is None:
            bond_list = self.get_bonds()
            
        total_market = sum([bond['current_fund_volume'] for bond in bond_list])
        underrate_market = sum([bond['current_fund_volume'] for bond in bond_list if bond['price'] <= UNDERRATE_PRICE])
        avg_price = mean([bond['price'] for bond in bond_list])
        avg_premium_ratio = mean([bond['convert_premium_ratio'] for bond in bond_list])
        return (total_market, underrate_market, avg_price, avg_premium_ratio)
                        
        

In [14]:
class StockBeta(object):
    
    def __init__(self, stock_code, index_type=0, base_date=None, history_days=365*5):
        """
        input:
            index_code: 要查询指数的代码
            index_type: 1为等权重方式计算，0为按市值加权计算
            base_date: 查询时间，格式为'yyyy-MM-dd'，默认为当天
            history_days: 默认历史区间位前八年
        """
        self._stock_code = stock_code
        self._index_type = index_type
        if not base_date:
            self._base_date = datetime.now().date() - timedelta(1)
        else:
            self._base_date = datetime.strptime(base_date, '%Y-%m-%d')
            
        self._begin_date = self._base_date - timedelta(history_days)
        self._end_date = self._base_date
        
        self._begin_date = self._begin_date.strftime('%Y-%m-%d')
        self._end_date = self._end_date.strftime('%Y-%m-%d')
        self._base_date = self._base_date.strftime('%Y-%m-%d')
            
    def get_stock_beta_factor(self, day=None):
        """
        获取当前时间的pe, pb值
        
        input:
            day: datetime.date类型，如果为None，默认代表取当前时间

        output:
            (pe, pb, roe, circulating_market_cap)
        """
        if not day:
            day = datetime.strptime(self._base_date, '%Y-%m-%d')
        
        stocks = [self._stock_code]
        q = query(
            valuation.pe_ratio, valuation.pb_ratio, valuation.circulating_market_cap
        ).filter(
            valuation.code.in_(stocks)
        )

        df = get_fundamentals(q, day)

        df = df[df['pe_ratio']>0]

        if len(df)>0:
            if(self._index_type == 0):
                pe = df['circulating_market_cap'].sum() / (df['circulating_market_cap']/df['pe_ratio']).sum()
                pb = df['circulating_market_cap'].sum() / (df['circulating_market_cap']/df['pb_ratio']).sum()
            else:
                pe = df['pe_ratio'].size / (1/df['pe_ratio']).sum()
                pb = df['pb_ratio'].size / (1/df['pb_ratio']).sum()
            return (pe, pb, pb/pe, df['circulating_market_cap'].sum() * pow(10, 8))
        else:
            return (None, None, None, None)
        
    def get_stock_beta_history_factors(self, interval=7):
        """
        获取任意指数一段时间的历史 pe,pb 估值列表，通过计算当前的估值在历史估值的百分位，来判断当前市场的估值高低。
        由于加权方式可能不同，可能公开的估值数据有差异，但用于判断估值相对高低没有问题

        input：
            interval: 计算指数估值的间隔天数，增加间隔时间可提高计算性能

        output：
            result:  指数历史估值的 DataFrame，index 为时间，列为pe，pb,roe
        """
        all_days = get_all_trade_days()

        pes = []
        roes = []
        pbs = []
        days = []

        begin = datetime.strptime(self._begin_date, '%Y-%m-%d').date()
        end = datetime.strptime(self._end_date, '%Y-%m-%d').date()  
        i = 0
        for day in all_days:
            if(day <= begin or day >= end):
                continue

            i += 1

            if(i % interval != 0):
                continue

            pe, pb, roe, circulating_market_cap = self.get_stock_beta_factor(day)
            if pe and pb and roe:
                pes.append(pe)
                pbs.append(pb)
                roes.append(roe)
                days.append(day)

        result = pd.DataFrame({'pe':pes,'pb':pbs, 'roe':roes}, index=days)
        return result
    
    def get_quantile_of_history_factors(self, factor, history_list):
        """
            获取某个因子在历史上的百分位，比如当前PE处于历史上的70%区间，意味着历史PE有70%都在当前值之下

        input:
            factor: beta因子
            history_list: 历史估值列表, DataFrame

        output:
            quantile: 历史估值百分位 (0.7)
        """
        factors = [history_list.quantile(i / 10.0)  for i in range(11)]    
        idx = bisect.bisect(factors, factor)
        if idx < 10:
            quantile = idx - (factors[idx] - factor) / (factors[idx] - factors[idx-1])   
            return quantile / 10.0    
        else:
            return 1.0


In [15]:
DOUBLE_LOW_VALUE = 130
DOUBLE_LOW_PRICE = 100

class DLowStrategy(object):
    
    # 双低策略转债个数
    EXPECTED_ITEMS_COUNT = 20
    
    def __init__(self, bond_list, base_date):
        self._bond_list = copy.deepcopy(bond_list)
        self._base_date = datetime.strptime(base_date, "%Y-%m-%d").date()
        
        
    def _set_stock_info(self, bond_list):
        """增加正股估值信息
        
            stock_pb: pb
            stock_pe: pe
            stock_pb_quantile: pb百分位
            stock_pe_quantile: pe百分位
        """
        for bond in bond_list:
            stock = StockBeta(bond['stock_code'])
            bond['stock_pe'], bond['stock_pb'], bond['stock_roe'], bond['circulating_market_cap'] = stock.get_stock_beta_factor()
            
            if bond['stock_pe'] is None or bond['stock_pb'] is None:
                # 正股报表有问题，剔除
                if bond in self._bond_list:
                    print("剔除_{}".format(bond))
                    self._bond_list.remove(bond)
                continue
            
            history_factors = stock.get_stock_beta_history_factors()
            bond['stock_pb_quantile'] = stock.get_quantile_of_history_factors(
                                                bond['stock_pb'], history_factors['pb'])
            bond['stock_pe_quantile'] = stock.get_quantile_of_history_factors(
                                                bond['stock_pe'], history_factors['pe'])
            bond['market_cap_ratio'] = bond['current_fund_volume'] / bond['circulating_market_cap'] * 100
            
    def _filter_last_cash_date(self, bond_list):
        """筛选掉最近到期项
        """
        filter_bond_list = filter(
            lambda x: x['last_cash_date'] - self._base_date > timedelta(360),
            bond_list
        )
        return list(filter_bond_list)
        
            
    def _filter_pb_pe_quantile(self, bond_list):
        """筛选pb, pe历史百分位
        """
        self._set_stock_info(bond_list)
        filter_bond_list = filter(
            lambda x: x['stock_pb'] > 1.3 and  x['stock_pe_quantile'] < 0.8,
            bond_list
        )
        return list(filter_bond_list)
        
    def _filter_pb(self, bond_list):
        """筛选PB>1.3防止下修转股价时破净限制
        """
        self._set_stock_info(bond_list)
        filter_bond_list = filter(
            lambda x: x.get('stock_pb', None) and x['stock_pb'] > 1.3,
            bond_list
        )
        return list(filter_bond_list)

        
        
    def _filter_current_fund_volume(self, bond_list):
        """剩余规模>1亿，且<10亿元的转债
        """
        filter_bond_list = filter(
            lambda x: x['current_fund_volume'] > pow(10, 8) and x['current_fund_volume'] < pow(10, 9),
            bond_list
        )
        return list(filter_bond_list)

    def _filter_current_market_volume(self, bond_list):
        """当日市场成交额>100万
        """
        filter_bond_list = filter(
            lambda x: x['day_market_volume'] > pow(10, 6),
            bond_list
        )
        return list(filter_bond_list)
        
    def _filter_convert_premium_ratio(self, bond_list):
        """溢价率小于30%
        """
        filter_bond_list = filter(
            lambda x: x['convert_premium_ratio'] < 0.3,
            bond_list
        )
        return list(filter_bond_list)
        
        
    def _filter_double_low(self, bond_list):
        """双低小于125-130
        """
        filter_bond_list = filter(
            lambda x: x['double_low'] < DOUBLE_LOW_VALUE,
            bond_list
        )
        return list(filter_bond_list)
    
    def _filter_price(self, bond_list):
        """价格过滤
        """
        filter_bond_list = filter(
            lambda x: x['price'] < DOUBLE_LOW_PRICE,
            bond_list
        )
        return list(filter_bond_list)   
    
    def _filter_stock_marketcap(self, bond_list):
        """过滤总市值<10亿或股价<3元的转债
        """        
        filter_bond_list = filter(
            lambda x: x['circulating_market_cap'] > pow(10, 9) and x['stock_price'] > 3,
            bond_list
        )        
        return list(filter_bond_list)           
        
    def _filter_strict(self, bond_list):
        """ROE>8%且转债占比<25%
        """
        filter_bond_list = filter(
            lambda x: x['stock_roe'] > 0.08 and x['market_cap_ratio'] < 25,
            bond_list
        )
                
        return list(filter_bond_list)    
               
        
    
    def get_support_bonds(self, filter_pb=False, filter_pb_pe_quantile=False, filter_price=False, filter_strict=False):
        """
        剩余规模>1亿，且<10亿元的转债
        
        到期时间>一年
        
        当前成交额>100万元的转债
        
        溢价率小于15%,主要是防止市场下跌时杀转债溢价；正股下跌，带动转债价格下跌；溢价率低的转债安全垫更厚；
        另外要注意折价的情况，最典型的就是2020年初的英联转债，折价转债是否值得入手，这个需要仔细研究，
        
        到期税后收益率大于0的转债
        
        取双低值<DOUBLE_LOW_VALUE的转债
        
        筛选PB>1.3防止下修转股价时破净限制
        
        filter_pb_pe_quantile=True, 筛选pb, pe历史百分位
        """
        support_bond_list = self._bond_list
        
        support_bond_list = self._filter_current_fund_volume(support_bond_list)
        support_bond_list = self._filter_current_market_volume(support_bond_list)
        support_bond_list = self._filter_convert_premium_ratio(support_bond_list)
        support_bond_list = self._filter_double_low(support_bond_list)
        support_bond_list = self._filter_last_cash_date(support_bond_list)
        
        
        if filter_pb:
            print("filter pb")
            support_bond_list = self._filter_pb(support_bond_list)
        
        if filter_pb_pe_quantile:
            print("filter pb quantile")
            support_bond_list = self._filter_pb_pe_quantile(support_bond_list)
            
        if filter_price:
            support_bond_list = self._filter_price(support_bond_list)                
            
        if filter_strict:
            print("市值>10亿，正股>3")
            support_bond_list = self._filter_stock_marketcap(support_bond_list)
            print("ROE>8%，转债占比<25%")
            support_bond_list = self._filter_strict(support_bond_list)                            
            
        return support_bond_list


In [16]:
# 测试
import pandas as pd
from datetime import datetime, timedelta
from jqfactor import *

import warnings

base_date = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')
#base_date = datetime.now()strftime('%Y-%m-%d')
# 后视镜转债历史低点  '2018-10-18' 
#base_date = '2018-10-18' 

index_bond = ConvertBondBeta(base_date=base_date)
print(base_date)
print("=========================")

# 获取指定时间转债列表及统计值 
bond_list = index_bond.get_bonds(base_date)
#pd.DataFrame(bond_list)

2021-02-09


In [17]:
# 采用价格过滤
bond_list_a = DLowStrategy(bond_list, base_date).get_support_bonds(filter_pb=False, filter_pb_pe_quantile=False, filter_price=True)
pd.DataFrame(bond_list_a)

,code,convert_premium_ratio,convert_price,current_fund_volume,day_market_volume,double_low,last_cash_date,price,raise_fund_volume,short_name,stock_code,stock_price
0,128093,0.053372,5.87,519584000.0,6.174526e+07,100.625205,2026-01-03,95.288,520000000.0,百川转债,002455.XSHE,5.31
1,113549,0.082350,8.88,879912000.0,4.857895e+06,102.575039,2025-11-15,94.340,880000000.0,白电转债,603861.XSHG,7.74
2,123063,0.067758,4.94,637936200.0,1.242169e+07,104.905815,2026-07-28,98.130,638000000.0,大禹转债,300021.XSHE,4.54
3,113568,0.164730,8.91,329934000.0,3.618523e+06,105.232990,2026-03-06,88.760,330000000.0,新春转债,603667.XSHG,6.79
4,113557,0.141608,10.11,599940000.0,4.067397e+06,105.850825,2025-12-19,91.690,600000000.0,森特转债,603098.XSHG,8.12
5,113565,0.093498,10.00,225428000.0,3.647517e+07,106.889776,2026-02-26,97.540,332000000.0,宏辉转债,603336.XSHG,8.92
6,110070,0.138086,2.75,288552000.0,3.104748e+06,107.338628,2026-04-13,93.530,440000000.0,凌钢转债,600231.XSHG,2.26
7,123049,0.095831,7.48,917056552.0,1.313675e+07,107.446120,2026-04-13,97.863,917240000.0,维尔转债,300190.XSHE,6.68
8,113578,0.205968,5.43,383923200.0,3.331643e+06,110.766823,2026-04-20,90.170,384000000.0,全筑转债,603030.XSHG,4.06
9,128087,0.298441,6.30,649870000.0,3.409409e+06,110.842122,2025-12-17,80.998,650000000.0,孚日转债,002083.XSHE,3.93


In [18]:
# 采用双低值过滤
bond_list_b = DLowStrategy(bond_list_a, base_date).get_support_bonds()
pd.DataFrame(bond_list_b)

,code,convert_premium_ratio,convert_price,current_fund_volume,day_market_volume,double_low,last_cash_date,price,raise_fund_volume,short_name,stock_code,stock_price
0,128093,0.053372,5.87,519584000.0,6.174526e+07,100.625205,2026-01-03,95.288,520000000.0,百川转债,002455.XSHE,5.31
1,113549,0.082350,8.88,879912000.0,4.857895e+06,102.575039,2025-11-15,94.340,880000000.0,白电转债,603861.XSHG,7.74
2,123063,0.067758,4.94,637936200.0,1.242169e+07,104.905815,2026-07-28,98.130,638000000.0,大禹转债,300021.XSHE,4.54
3,113568,0.164730,8.91,329934000.0,3.618523e+06,105.232990,2026-03-06,88.760,330000000.0,新春转债,603667.XSHG,6.79
4,113557,0.141608,10.11,599940000.0,4.067397e+06,105.850825,2025-12-19,91.690,600000000.0,森特转债,603098.XSHG,8.12
5,113565,0.093498,10.00,225428000.0,3.647517e+07,106.889776,2026-02-26,97.540,332000000.0,宏辉转债,603336.XSHG,8.92
6,110070,0.138086,2.75,288552000.0,3.104748e+06,107.338628,2026-04-13,93.530,440000000.0,凌钢转债,600231.XSHG,2.26
7,123049,0.095831,7.48,917056552.0,1.313675e+07,107.446120,2026-04-13,97.863,917240000.0,维尔转债,300190.XSHE,6.68
8,113578,0.205968,5.43,383923200.0,3.331643e+06,110.766823,2026-04-20,90.170,384000000.0,全筑转债,603030.XSHG,4.06
9,128087,0.298441,6.30,649870000.0,3.409409e+06,110.842122,2025-12-17,80.998,650000000.0,孚日转债,002083.XSHE,3.93


In [19]:
# 采用pb>1.3, pe, pb百分位筛选
bond_list_c = DLowStrategy(bond_list_a, base_date).get_support_bonds(filter_pb=True, filter_pb_pe_quantile=True)
pd.DataFrame(bond_list_c)


filter pb
剔除_{'code': '128082', 'short_name': '华锋转债', 'stock_code': '002806.XSHE', 'convert_price': 11.71, 'last_cash_date': datetime.date(2025, 12, 4), 'raise_fund_volume': 352400000.0, 'current_fund_volume': 344964360.0, 'price': 88.87, 'day_market_volume': 2067808.76, 'stock_price': 8.3, 'convert_premium_ratio': 0.25381650602409633, 'double_low': 114.25165060240964, 'stock_pe': None, 'stock_pb': None, 'stock_roe': None, 'circulating_market_cap': None}
filter pb quantile


,circulating_market_cap,code,convert_premium_ratio,convert_price,current_fund_volume,day_market_volume,double_low,last_cash_date,market_cap_ratio,price,raise_fund_volume,short_name,stock_code,stock_pb,stock_pb_quantile,stock_pe,stock_pe_quantile,stock_price,stock_roe
0,2.943860e+09,123063,0.067758,4.94,637936200.0,1.242169e+07,104.905815,2026-07-28,21.670059,98.130,638000000.0,大禹转债,300021.XSHE,2.5079,0.074613,38.6071,0.262662,4.54,0.064960
1,3.897760e+09,113557,0.141608,10.11,599940000.0,4.067397e+06,105.850825,2025-12-19,15.391917,91.690,600000000.0,森特转债,603098.XSHG,1.8803,0.008949,22.9312,0.292423,8.12,0.081997
2,3.009630e+09,113565,0.093498,10.00,225428000.0,3.647517e+07,106.889776,2026-02-26,7.490223,97.540,332000000.0,宏辉转债,603336.XSHG,3.2477,0.034021,38.3862,0.043124,8.92,0.084606
3,6.280970e+09,128083,0.168842,11.70,876912300.0,2.205084e+07,114.787194,2025-12-12,13.961415,97.903,877000000.0,新北转债,002376.XSHE,1.9327,0.064605,28.9974,0.408120,9.80,0.066651
4,4.506120e+09,128071,0.163738,4.28,299960125.0,1.140783e+07,115.617786,2025-08-16,6.656727,99.244,595750000.0,合兴转债,002228.XSHE,1.3528,0.024222,15.5829,0.280026,3.65,0.086813
5,1.464450e+09,113570,0.263644,11.54,279972000.0,1.717797e+06,116.374404,2026-03-11,19.117894,90.010,280000000.0,百达转债,603331.XSHG,1.8991,0.000890,27.0390,0.118899,8.22,0.070236
6,4.600630e+09,123044,0.236441,18.80,549607500.0,1.305829e+08,118.942058,2026-03-12,11.946353,95.298,585000000.0,红相转债,300427.XSHE,2.2953,0.145472,19.9363,0.155341,14.49,0.115132
7,1.264230e+09,123045,0.264366,20.13,166089450.0,2.752337e+07,121.656583,2026-03-12,13.137598,95.220,288500000.0,雷迪转债,300652.XSHE,1.7773,0.000660,25.4984,0.348167,15.16,0.069702
8,1.670800e+09,113561,0.280589,10.23,150162000.0,1.053979e+07,122.068895,2025-12-31,8.987431,94.010,290000000.0,正裕转债,603089.XSHG,1.8073,0.014509,48.2654,0.774810,7.51,0.037445


In [20]:
# 采用最严格的过滤方法: ROE>8%，转债占流通市值<30%，
bond_list_d = DLowStrategy(bond_list_a, base_date).get_support_bonds(filter_pb=True, filter_pb_pe_quantile=True, filter_strict=True)
pd.DataFrame(bond_list_d)

filter pb
剔除_{'code': '128082', 'short_name': '华锋转债', 'stock_code': '002806.XSHE', 'convert_price': 11.71, 'last_cash_date': datetime.date(2025, 12, 4), 'raise_fund_volume': 352400000.0, 'current_fund_volume': 344964360.0, 'price': 88.87, 'day_market_volume': 2067808.76, 'stock_price': 8.3, 'convert_premium_ratio': 0.25381650602409633, 'double_low': 114.25165060240964, 'stock_pe': None, 'stock_pb': None, 'stock_roe': None, 'circulating_market_cap': None}
filter pb quantile
市值>10亿，正股>3
ROE>8%，转债占比<25%


,circulating_market_cap,code,convert_premium_ratio,convert_price,current_fund_volume,day_market_volume,double_low,last_cash_date,market_cap_ratio,price,raise_fund_volume,short_name,stock_code,stock_pb,stock_pb_quantile,stock_pe,stock_pe_quantile,stock_price,stock_roe
0,3.897760e+09,113557,0.141608,10.11,599940000.0,4.067397e+06,105.850825,2025-12-19,15.391917,91.690,600000000.0,森特转债,603098.XSHG,1.8803,0.008949,22.9312,0.292423,8.12,0.081997
1,3.009630e+09,113565,0.093498,10.00,225428000.0,3.647517e+07,106.889776,2026-02-26,7.490223,97.540,332000000.0,宏辉转债,603336.XSHG,3.2477,0.034021,38.3862,0.043124,8.92,0.084606
2,4.506120e+09,128071,0.163738,4.28,299960125.0,1.140783e+07,115.617786,2025-08-16,6.656727,99.244,595750000.0,合兴转债,002228.XSHE,1.3528,0.024222,15.5829,0.280026,3.65,0.086813
3,4.600630e+09,123044,0.236441,18.80,549607500.0,1.305829e+08,118.942058,2026-03-12,11.946353,95.298,585000000.0,红相转债,300427.XSHE,2.2953,0.145472,19.9363,0.155341,14.49,0.115132


In [21]:
bond_list_a_text = pd.DataFrame(bond_list_a).to_html()
bond_list_b_text = pd.DataFrame(bond_list_b).to_html()
bond_list_c_text = pd.DataFrame(bond_list_c).to_html()
bond_list_d_text = pd.DataFrame(bond_list_d).to_html()


# 取得几个统计值：市场总量，小于110元转债总量，价格算术平均值，溢价率算术平均值
total_market, underrate_market, avg_price, avg_premium_ratio = index_bond.get_bonds_factors(bond_list)
total_market_text = "市场总量{}亿元, 小于110元转债总量{}亿元, 价格算术平均值{}, 溢价率算术平均值{}".format(
                                                                total_market/100000000,
                                                                underrate_market/100000000,
                                                                avg_price, avg_premium_ratio
                    )

split_text = '=================================================='
print(total_market_text)

send_message_text = "{}\r\n{}\r\n{}\r\n{}\r\n{}\r\n{}\r\n".format(
    bond_list_a_text, split_text, bond_list_b_text, bond_list_c_text, split_text, total_market_text)
#print(send_message_text)

市场总量1216.6618263870998亿元, 小于110元转债总量956.5715896670999亿元, 价格算术平均值142.0083009708738, 溢价率算术平均值0.2274989196397154
